# Tweet Classification

## Data Set Loading

### DATA Load

In [1]:
import pandas as pd

In [2]:
# Concatenate social spambot tweets
bot_tweets = pd.concat([
    pd.read_csv('D:/DNNforBotDetection/DNNforBotDetection/input/cresci-2017.csv/Tweets/Social_spambots_1.csv', encoding='latin-1'),
    pd.read_csv('D:/DNNforBotDetection/DNNforBotDetection/input/cresci-2017.csv/Tweets/social_spambots_2.csv', encoding='latin-1'),
    pd.read_csv('D:/DNNforBotDetection/DNNforBotDetection/input/cresci-2017.csv/Tweets/social_spambots_3.csv', encoding='latin-1')
]).reset_index(drop=True)

# Read genuine account tweets
clean_tweets = pd.read_csv('D:/DNNforBotDetection/DNNforBotDetection/input/cresci-2017.csv/Tweets/genuine_accounts.csv', encoding='latin-1')


C:\Users\hassa\AppData\Local\Temp\ipykernel_9036\2020695786.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('D:/DNNforBotDetection/DNNforBotDetection/input/cresci-2017.csv/Tweets/Social_spambots_1.csv', encoding='latin-1'),
C:\Users\hassa\AppData\Local\Temp\ipykernel_9036\2020695786.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('D:/DNNforBotDetection/DNNforBotDetection/input/cresci-2017.csv/Tweets/social_spambots_2.csv', encoding='latin-1'),
C:\Users\hassa\AppData\Local\Temp\ipykernel_9036\2020695786.py:5: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('D:/DNNforBotDetection/DNNforBotDetection/input/cresci-2017.csv/Tweets/social_spambots_3.csv', encoding='latin-1')
C:\Users\hassa\AppData\Local\Temp\ipykernel_9036\2020695786.py:9: DtypeWarning: Columns (0) have mixed types

In [3]:
print(bot_tweets.dtypes)

id                           int64
text                        object
source                      object
user_id                      int64
truncated                  float64
in_reply_to_status_id        int64
in_reply_to_user_id          int64
in_reply_to_screen_name     object
retweeted_status_id          int64
geo                        float64
place                       object
contributors               float64
retweet_count                int64
reply_count                  int64
favorite_count               int64
favorited                  float64
retweeted                  float64
possibly_sensitive         float64
num_hashtags                 int64
num_urls                     int64
num_mentions                 int64
created_at                  object
timestamp                   object
crawled_at                  object
updated                     object
dtype: object


### Data Pre-Processing

In [4]:
import re
import emoji
import pandas as pd

In [5]:
# List of columns to drop
columns_to_drop = ['truncated', 'in_reply_to_status_id', 'in_reply_to_user_id', 'in_reply_to_screen_name',
                   'retweeted_status_id', 'geo', 'place', 'contributors', 'favorited', 'retweeted',
                   'possibly_sensitive', 'created_at', 'timestamp', 'crawled_at', 'updated']

# Drop the columns from the DataFrame
bot_tweets = bot_tweets.drop(columns=columns_to_drop)
clean_tweets = clean_tweets.drop(columns=columns_to_drop)

In [6]:
# Drop 85% of samples to make it computationally feasible

clean_tweets = clean_tweets.drop(clean_tweets.sample(frac=0.5, random_state=42).index)
bot_tweets = bot_tweets.drop(bot_tweets.sample(frac=0.5, random_state=42).index)

In [7]:
bot_tweets.dtypes

id                 int64
text              object
source            object
user_id            int64
retweet_count      int64
reply_count        int64
favorite_count     int64
num_hashtags       int64
num_urls           int64
num_mentions       int64
dtype: object

In [8]:
bot_tweets['BotOrNot'] = 1
clean_tweets['BotOrNot'] = 0

Tweets1_df = pd.concat([bot_tweets, clean_tweets])

Tweets_df = Tweets1_df.sample(frac=1).reset_index(drop=True)

In [9]:
# Remove null samples from the 'text' column
Tweets_df = Tweets_df.dropna(subset=['text'])

In [10]:
Tweets_df

,id,text,source,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,BotOrNot
0,461800985719885824,Russia invaded and controlled Ukraine without ...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2.384166e+09,0.0,0.0,0.0,0.0,0.0,0.0,1
1,525750960510222336,Magari la vita fosse facile come incontrare ch...,"<a href=""http://www.fedeconiglietta.com"" rel=""...",4.663751e+08,0.0,0.0,0.0,0.0,0.0,0.0,1
2,510649702099202048,Yum! Even MORE Brownie's Edge In This Baker's ...,"<a href=""http://tweetadder.com"" rel=""nofollow""...",3.354013e+08,0.0,0.0,0.0,0.0,1.0,0.0,1
3,514068776598659072,Check Out The Top 10 European Beers http://t....,"<a href=""http://tweetadder.com"" rel=""nofollow""...",1.190159e+09,0.0,0.0,0.0,0.0,1.0,0.0,1
4,567891364337160192,RT @eternalquote_: Emotionally: I'm done. Ment...,"<a href=""http://twitter.com/download/iphone"" r...",1.919095e+09,954.0,0.0,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...
3148243,409271432980725760,"Tutti gli uomini sono commedianti. Salvo, fors...","<a href=""http://www.giovannamezzett.it"" rel=""n...",1.273201e+09,0.0,0.0,0.0,0.0,0.0,0.0,1
3148244,585241732457095168,RT @tomkrazit: 2015. https://t.co/q244TbZYit,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",1.430704e+07,14.0,0.0,0.0,0.0,1.0,1.0,0
3148245,568779099172442112,@Celnide https://t.co/44xxim7eMq,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",5.677650e+08,1.0,0.0,1.0,0.0,1.0,1.0,0
3148246,460960090019749888,"They were right, 11 hours of sleep does make a...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2.359594e+09,0.0,0.0,0.0,0.0,0.0,0.0,1


In [11]:
def replace_hashtags(tweet):
    return re.sub(r'#\w+', '<hashtag>', tweet)

def replace_urls(tweet):
    return re.sub(r'http\S+|www\S+', '<url>', tweet)

def replace_numbers(tweet):
    return re.sub(r'\d+', '<number>', tweet)

def replace_user_mentions(tweet):
    return re.sub(r'@\w+', '<user>', tweet)

def replace_emojis(tweet):
    emoji_dict = {
        ":)": "<smile>",
        "<3": "<heart>",
        ":D": "<lolface>",
        ":|": "<neutralface>",
        ">:(": "<angryface>"
    }
    for emoticon, tag in emoji_dict.items():
        tweet = tweet.replace(emoticon, tag)
    tweet = emoji.demojize(tweet)
    tweet = re.sub(r':\w+:', '', tweet)
    return tweet


In [12]:
def add_allcaps_tag(word):
    if word.isupper() or re.search(r'(\w)\1{2,}', word):
        return f"{word} <allcaps>"
    return word


In [13]:
def preprocess_tweets(df):
    df['text'] = df['text'].str.lower()
    df['text'] = df['text'].apply(replace_hashtags)
    df['text'] = df['text'].apply(replace_urls)
    df['text'] = df['text'].apply(replace_numbers)
    df['text'] = df['text'].apply(replace_user_mentions)
    df['text'] = df['text'].apply(replace_emojis)
    df['text'] = df['text'].apply(lambda tweet: ' '.join(add_allcaps_tag(word) for word in tweet.split()))
    return df


In [14]:
Final_Tweet_df = preprocess_tweets(Tweets_df)

In [15]:
Final_Tweet_df.columns

Index(['id', 'text', 'source', 'user_id', 'retweet_count', 'reply_count',
       'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions',
       'BotOrNot'],
      dtype='object')

### GLove model for creating vector embeddings

In [16]:
import numpy as np
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

In [17]:
from gensim.models import KeyedVectors

glove_path = "D:/DNNforBotDetection/DNNforBotDetection/input/glove.6B.50d.txt"  # Path to the downloaded GloVe file

word_vectors = KeyedVectors.load_word2vec_format(glove_path, binary=False, no_header=True)


In [18]:
def get_tweet_embedding(tokens):
    embeddings = []
    for token in tokens:
        if token in word_vectors:
            embeddings.append(word_vectors[token])
    if embeddings:
        return np.mean(embeddings, axis=0)  # Average the word embeddings to get the tweet embedding
    return np.zeros(word_vectors.vector_size)  # Return a zero vector if no valid word embeddings found


In [19]:
Final_Tweet_df['embedding'] = Final_Tweet_df['text'].apply(get_tweet_embedding)

In [20]:
Final_Tweet_df

,id,text,source,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,BotOrNot,embedding
0,461800985719885824,russia invaded and controlled ukraine without ...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2.384166e+09,0.0,0.0,0.0,0.0,0.0,0.0,1,"[-0.07598507, 0.75142807, 0.5416522, 0.5671289..."
1,525750960510222336,magari la vita fosse facile come incontrare ch...,"<a href=""http://www.fedeconiglietta.com"" rel=""...",4.663751e+08,0.0,0.0,0.0,0.0,0.0,0.0,1,"[-0.055632707, 0.7434381, 0.46346405, 0.530804..."
2,510649702099202048,yum! even more brownie's edge in this baker's ...,"<a href=""http://tweetadder.com"" rel=""nofollow""...",3.354013e+08,0.0,0.0,0.0,0.0,1.0,0.0,1,"[-0.06478623, 0.8711866, 0.7463261, 0.65168756..."
3,514068776598659072,check out the top <number> european beers <url>,"<a href=""http://tweetadder.com"" rel=""nofollow""...",1.190159e+09,0.0,0.0,0.0,0.0,1.0,0.0,1,"[-0.1301706, 0.8546163, 0.86320895, 0.8093027,..."
4,567891364337160192,rt <user>: emotionally: i'm done. mentally: i'...,"<a href=""http://twitter.com/download/iphone"" r...",1.919095e+09,954.0,0.0,0.0,0.0,0.0,1.0,0,"[-0.042735964, 0.8399097, 0.5665991, 0.5889838..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3148243,409271432980725760,"tutti gli uomini sono commedianti. salvo, fors...","<a href=""http://www.giovannamezzett.it"" rel=""n...",1.273201e+09,0.0,0.0,0.0,0.0,0.0,0.0,1,"[-0.10696832, 0.8045218, 0.5142709, 0.63387585..."
3148244,585241732457095168,rt <user>: <number>. <url>,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",1.430704e+07,14.0,0.0,0.0,0.0,1.0,1.0,0,"[-0.28160304, 0.9147292, 0.8585943, 0.7072718,..."
3148245,568779099172442112,<user> <url>,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",5.677650e+08,1.0,0.0,1.0,0.0,1.0,1.0,0,"[-0.31176457, 0.9354463, 0.97580546, 0.7138443..."
3148246,460960090019749888,"they were right, <number> hours of sleep does ...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2.359594e+09,0.0,0.0,0.0,0.0,0.0,0.0,1,"[-0.042039264, 0.88863474, 0.7665115, 0.797695..."


In [21]:
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense, concatenate, Dropout
from sklearn.model_selection import train_test_split
from keras.utils import pad_sequences
import tensorflow as tf
from keras.optimizers import Adam, SGD

# Assuming you have the following variables with your tweet data
tokenized_tweets = Final_Tweet_df['embedding']
metadata = Final_Tweet_df[['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions']]
labels = Final_Tweet_df['BotOrNot']

# Convert tokenized tweets to sequences
sequences = tokenized_tweets

# Pad sequences to have the same length
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)
#padded_sequences = padded_sequences.reshape(padded_sequences.shape[0], padded_sequences.shape[1], 1)
#padded_sequences = np.expand_dims(padded_sequences, axis=-1)


# Split data into train and test sets
X_train_text, X_test_text, X_train_metadata, X_test_metadata, y_train, y_test = train_test_split(
    padded_sequences, metadata, labels, test_size=0.2, random_state=42)

# Define LSTM model architecture
tweet_input = Input(shape=(max_sequence_length,))
metadata_input = Input(shape=(X_train_metadata.shape[1],))
tweet_input = tf.expand_dims(tweet_input, axis=-1)

Optimizer = Adam(learning_rate=0.0007)  # Specify your desired learning rate here

lstm_output = LSTM(32)(tweet_input)

concatenated_output = concatenate([lstm_output, metadata_input])
dense_layer = Dense(128, activation='relu')(concatenated_output)
dense_layer = Dropout(0.2)(dense_layer)

dense_layer2 = Dense(64, activation='relu')(concatenated_output)
dense_layer2 = Dropout(0.1)(dense_layer)

output = Dense(1, activation='sigmoid')(dense_layer)

model = Model(inputs=[tweet_input, metadata_input], outputs=output)
model.compile(loss='binary_crossentropy', optimizer=Optimizer, metrics=['accuracy'])

# Train the model
model.fit([X_train_text, X_train_metadata], y_train, batch_size=256, epochs=20, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate([X_test_text, X_test_metadata], y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

Epoch 1/20


C:\Users\hassa\.conda\envs\NLP\lib\site-packages\keras\engine\data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


7855/7855 [==============================] - 118s 15ms/step - loss: 0.7648 - accuracy: 0.7951 - val_loss: 0.5105 - val_accuracy: 0.8024
Epoch 2/20
7855/7855 [==============================] - 116s 15ms/step - loss: 0.5046 - accuracy: 0.8019 - val_loss: 0.4680 - val_accuracy: 0.8046
Epoch 3/20
7855/7855 [==============================] - 119s 15ms/step - loss: 0.4749 - accuracy: 0.8028 - val_loss: 0.4681 - val_accuracy: 0.8046
Epoch 4/20
7855/7855 [==============================] - 120s 15ms/step - loss: 0.4706 - accuracy: 0.8041 - val_loss: 0.4627 - val_accuracy: 0.8061
Epoch 5/20
7855/7855 [==============================] - 122s 15ms/step - loss: 0.4706 - accuracy: 0.8046 - val_loss: 0.4620 - val_accuracy: 0.8064
Epoch 6/20
7855/7855 [==============================] - 123s 16ms/step - loss: 0.4708 - accuracy: 0.8048 - val_loss: 0.4647 - val_accuracy: 0.8046
Epoch 7/20
7855/7855 [==============================] - 124s 16ms/step - loss: 0.4682 - accuracy: 0.8047 - val_loss: 0.4645 - val

In [22]:
model.save('tweet_model.h5')

In [25]:
from sklearn.metrics import precision_score, roc_auc_score, recall_score, f1_score
# Evaluating the model
y_pred_prob = model.predict([X_test_text, X_test_metadata])
y_pred = (y_pred_prob >= 0.5).astype(int)
y_scores = y_pred_prob.ravel()

recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_scores)

print("Recall Score:", recall)
print("F1 Score:", f1)
print("ROC/AUC Score:", roc_auc)

19636/19636 [==============================] - 42s 2ms/step
Recall Score: 0.8872471207100318
F1 Score: 0.834560110730113
ROC/AUC Score: 0.8468545091803743
